In [ ]:
!pip install datasets dataclasses transformers

In [1]:
# Load the Iris dataset from sklearn
from sklearn.datasets import load_iris
import pandas as pd

# Load the dataset
iris = load_iris()

# Convert to a pandas DataFrame
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['species'] = iris.target

# Map target values to species names
data['species'] = data['species'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

# Display the first few rows of the dataset
print(data.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

  species  
0  setosa  
1  setosa  
2  setosa  
3  setosa  
4  setosa  


In [2]:
# Unzip the uploaded tabula.zip file
import zipfile
import os

# Define the path for the uploaded zip file
zip_path = "tabula.zip"

# Unzip the file into the current working directory
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("tabula_repo")

# Change directory to the extracted folder
os.chdir("tabula_repo")

# Verify the contents of the extracted folder
!ls

tabula


In [3]:
from tabula import Tabula

In [4]:
# Define categorical columns
categorical_columns = ['species']

# Initialize Tabula
model = Tabula(
    llm='distilgpt2',
    experiment_dir='iris_synthetic_data',
    batch_size=8,
    epochs=10,
    categorical_columns=categorical_columns
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
model.fit(data)

/content/tabula_repo/tabula/tabula.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  tabula_trainer = TabulaTrainer(self.model, training_args, train_dataset=tabula_ds, tokenizer=self.tokenizer,
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [6]:
import torch
torch.save(model.model.state_dict(), "iris_model_10epoch.pt")

In [8]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Set the pad_token to a unique value (e.g., "<|pad|>")
tokenizer.add_special_tokens({'pad_token': '<|pad|>'})

# Update the model with the tokenizer's new configuration
model.model.resize_token_embeddings(len(tokenizer))
model.model.config.pad_token_id = tokenizer.pad_token_id

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [10]:
from transformers import AutoTokenizer

# Load and adjust the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.add_special_tokens({'pad_token': '<|pad|>'})

# Update the model with the new tokenizer configuration
model.model.resize_token_embeddings(len(tokenizer))
model.model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
# Retry sampling
synthetic_data = model.sample(n_samples=33)
synthetic_data.to_csv("iris_10epoch.csv", index=False)

  0%|          | 0/33 [07:42<?, ?it/s]